In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import sys

from sklearn.datasets import load_diabetes
sys.path.append("..")
from interpretDistill.fourierDistill import FTDistill, FTDistillCV
from interpretDistill.binaryTransformer import BinaryTransformer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from interpretDistill.FTutils import *

In [2]:
X, y = load_diabetes(as_frame = True, return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [3]:
bt = BinaryTransformer(depth=1)
#X_train = bt.fit_and_transform(X_train, y_train)
#X_val = bt.transform(X_val)
X_val = bt.fit_and_transform(X_val, y_val)
X_train = bt.transform(X_train)
X_test = bt.transform(X_test)

In [4]:
rf = RandomForestRegressor(max_depth=4, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, random_state=42)

In [5]:
X_train.shape, X_val.shape, X_test.shape

((176, 19), (177, 19), (89, 19))

In [6]:
r2_score(y_train, rf.predict(X_train)), r2_score(y_val, rf.predict(X_val)), r2_score(y_test, rf.predict(X_test))

(0.5690328686530103, 0.45458549134785586, 0.3327758359638333)

In [14]:
#ftd = FTDistillCV(selection = 'L1L2', lam1_range=np.arange(0.01, 1.1, 0.33), lam2_range=np.arange(0.01, 1.1, 0.33), size_interactions = 3)
ftd = FTDistill(selection = 'L1L2', lam1=15, lam2=15, size_interactions = 6)
ftd.fit(X_val, rf.predict(X_val))

In [15]:
np.sum(ftd.regression_model.coef_ != 0)

468

In [9]:
r2_score(y_train, ftd.predict(X_train)), r2_score(y_val, ftd.predict(X_val)), r2_score(y_test, ftd.predict(X_test))

(-0.8625133699900722, -0.8707487057230432, -0.721514871233137)

In [16]:
#ftd = FTDistillCV(selection = 'L1L2', lam1_range=np.arange(0.01, 1.1, 0.33), lam2_range=np.arange(0.01, 1.1, 0.33), size_interactions = 3)
ftd = FTDistill(selection = 'L1L2', lam1=15, lam2=15, size_interactions = 6)
ftd.fit(X_train, rf.predict(X_train))

In [17]:
np.sum(ftd.regression_model.coef_ != 0)

816

In [18]:
r2_score(y_train, ftd.predict(X_train)), r2_score(y_val, ftd.predict(X_val)), r2_score(y_test, ftd.predict(X_test))

(0.13956876899349424, 0.12958447111020788, 0.19794941311093572)

In [13]:
bt = BinaryTransformer(depth=1)
binary = bt.fit_and_transform(X, y)
#ftd = FTDistillCV(selection = 'L1L2', lam1_range=np.arange(0.01, 1.1, 0.33), lam2_range=np.arange(0.01, 1.1, 0.33), size_interactions = 4)
ftd = FTDistill(selection = 'L1L2', lam1=0.1, lam2=0.1, size_interactions = 4)
ftd.fit(binary, y, bt.no_interaction)

TypeError: fit() takes from 2 to 3 positional arguments but 4 were given

In [ ]:
np.mean(ftd.regression_model.coef_ != 0)

In [ ]:
bt.no_interaction

In [ ]:
np.sum(ftd_naive.regression_model.coef_ != 0)

In [ ]:
np.array(list(map(str, ftd_naive.features)))[ftd_naive.regression_model.coef_ != 0]

In [ ]:
np.sum(binary.values == 1) + np.sum(binary.values == -1) == np.prod(binary.shape)

In [ ]:
import pandas as pd

# Initialize memoization dictionary
memo = {}

def compute_subset_product(subset, data):
    global memo
    
    if subset in memo:
        print("Using memoized result for subset:", subset)
        return memo[subset]
    
    result = pd.Series(1, index=data.index) if not subset else _compute_subset_product_helper(subset, data)
    
    memo[subset] = result
    
    return result

def _compute_subset_product_helper(subset, data):
    global memo
    
    if len(subset) == 1:
        print("Computed product for subset:", subset)
        return data[subset[0]]
    
    if subset in memo:
        return memo[subset]
    
    if set(subset) in [set(k) for k in memo.keys()]:
        
        shuffled_permutations = itertools.permutations(subset)

        for perm in shuffled_permutations:
            if perm in my_dict:
                return memo[perm]
    
    overlaps = [(k, len(set(subset).intersection(set(k)))) for k in memo.keys()]
    print(f'overlaps: {overlaps} for subset: {subset}')
    if len(overlaps) == 0 or max(overlaps, key=lambda x: x[1])[1] == 0:
        result = pd.Series(1, index = data.index) if not subset else data[list(subset)].product(axis=1)
        memo[subset] = result
        return result
    else:
        max_subset = max(overlaps, key=lambda x: x[1])[0]
        max_subset_product = memo[max_subset]
        remaining_subset = tuple(set(subset) - set(max_subset))

        remaining_subset_product = _compute_subset_product_helper(remaining_subset, data)
        result = max_subset_product * remaining_subset_product

        memo[subset] = result

        return result
        

# Example usage
data = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9], 'D': [10, 11, 12], 'E': [9, 8, 7]})
subset = [('A', 'B'), ('C', 'D', 'E'), ('A', 'B', 'C', 'D', 'E')]
Chi = pd.DataFrame()
for s in subset:
    Chi[s] = compute_subset_product(s, data)

In [ ]:
Chi

In [ ]:
memo = {('a', 'b'): 'hello'}

In [ ]:
memo[('b', 'a')]

In [ ]:
t = [(1), (2, 3), (), (5, 6, 7)]

In [ ]:
sorted(t, key=len)

In [ ]:
list_of_tuples = [(), (1,), (1, 2), (1, 2, 3), (1, 2, 3, 4), (1, 2, 3, 4, 5)]

sorted_list = sorted(list_of_tuples, key=len)

print("Sorted list:", sorted_list)